# Feature Engineering - Neighborhoods

In [155]:
# installing the requirements:
!pip install -r ../configs/dependencies/dataprep_requirements.txt >> ../configs/dependencies/package_installation.txt

In [156]:
# loading the magic commands:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
###### Loading the necessary libraries #########

# PySpark dependencies:s
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
import pyspark.sql.types as T
from pyspark.sql.window import Window

# Sedona dependencies:
from sedona.utils.adapter import Adapter
from sedona.register import SedonaRegistrator
from sedona.utils import KryoSerializer, SedonaKryoRegistrator
from sedona.core.SpatialRDD import SpatialRDD
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import GeoJsonReader

# database utilities:
from sqlalchemy import create_engine
import sqlite3 as db
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import fiona

# plotting and data visualization:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, Image

# other relevant libraries:
import warnings
import unidecode
import inflection
import unicodedata
from datetime import datetime, timedelta
from functools import partial
import json
import re
import os
from glob import glob
import shutil
import itertools
import chardet

# importing the atlas utilities:
from atlasutils import (
    save_to_filesystem,
    save_as_table,
    rotate_xticks,
    get_file_encoding,
    normalize_entities,
    normalize_column_name,
    apply_category_map,
    standardize_variable_names,
    get_null_columns,
    replace_decimal_separator,
    convert_to_geopandas,
    drop_invalid_census_columns,
    clean_census_column_name,
    get_file_crs,
    get_column_values,
    load_geospatial_file,
    convert_geometry,
    rank_feature,
    save_geospatial_file,
)


# setting global parameters for visualizationsss:
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

# 0. Configuring Spark

In [158]:
# function to encapsulate standard spark configurations:
def init_spark(app_name):

    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.files.overwrite", "true")
        .config("spark.serializer", KryoSerializer.getName)
        .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)
        .config(
            "spark.jars.packages",
            "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.1-incubating,"
            "org.datasyslab:geotools-wrapper:geotools-24.1",
        )
        .config("spark.sql.repl.eagerEval.enabled", True)
        .config("spark.sql.repl.eagerEval.maxNumRows", 5)
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
        .config("spark.sql.parquet.compression.codec", "gzip")
        .config("sedona.global.charset", "utf8")
        .config("sedona.global.index", "true")
        .enableHiveSupport()
        .getOrCreate()
    )

    SedonaRegistrator.registerAll(spark)

    return spark

In [164]:
# init the spark session:
spark = init_spark("SP Atlas - Neighborhood-level Feature Engineering")

In [165]:
# verifying the session status:
spark

# 1. Loading and Inspecting the Data

## 1.1 Additional Layers

In [166]:
# loading the raw dataset:
LAYERS_DATA_DIR = "../data/raw/sp_layers/"
RAW_DATA_DIR = "../data/raw/"

### 1.1.1 Parks

In [167]:
# loading the park data:
df_parks = load_geospatial_file(LAYERS_DATA_DIR + "parks/*", spark)

# convert the geometry type:
df_parks = convert_geometry(df_parks, "geometry", "epsg:29193", "epsg:4326")

In [168]:
# improving column names:
df_parks = df_parks.select(
    F.col("pq_id").alias("park_id"),
    F.col("pq_area").alias("park_area"),
    F.col("pq_nome").alias("park_name"),
    F.col("pq_PrefReg").alias("park_subprefecture"),
    F.col("geometry"),
)

### 1.1.2 Favelas

In [169]:
# loading the park data:
df_favelas = load_geospatial_file(LAYERS_DATA_DIR + "favelas_macro/*", spark)

# convert the geometry type:
df_favelas = convert_geometry(df_favelas, "geometry", "epsg:29193", "epsg:4326")

In [170]:
# fixing the columns:
new_cols = ["geometry", "favela_id", "favela_area", "favela_name"]

df_favelas = df_favelas.drop("DATULTATZ", "ENDERECO", "NOMESEC", "PROPRTR")

original_cols = df_favelas.columns
cols_map = dict(zip(original_cols, new_cols))

for col, new_col in cols_map.items():
    df_favelas = df_favelas.withColumnRenamed(col, new_col)

### 1.1.3 Train and Metro

In [171]:
df_train = load_geospatial_file(LAYERS_DATA_DIR + "train/*", spark)
df_train = convert_geometry(df_train, "geometry", "epsg:29193", "epsg:4326")

# cleaning columns:
df_train = df_train.select(F.col("geometry"), F.col("etr_nome").alias("station_name"))

In [172]:
df_metro = load_geospatial_file(LAYERS_DATA_DIR + "metro/*", spark)
df_metro = convert_geometry(df_metro, "geometry", "epsg:29193", "epsg:4326")

# cleaning columns:
df_metro = df_metro.select(F.col("geometry"), F.col("emt_nome").alias("station_name"))

In [173]:
# union all dataframes:
df_rail = df_train.union(df_metro)

In [174]:
# verifying the number of resulting records:
df_rail.count()

196

## 1.2 Neighborhood Features

In [177]:
# Data prep results:
PROCESSED_DATA_DIR = "../data/processed/"

# reading the result from the data prep:
df_neighborhood_census = spark.read.parquet(
    PROCESSED_DATA_DIR
    + "sp_census/units_of_interest/neighborhoods/tb_neighborhood_census.parquet"
)

df_neighborhood_iptu = spark.read.parquet(
    PROCESSED_DATA_DIR + "sp_iptu/tb_neighborhood_iptu_no_geo.parquet"
)

In [182]:
# joinining the district base features:
df_neighborhood = df_neighborhood_census.join(
    df_neighborhood_iptu,
    how="left",
    on=(df_neighborhood_census.neighborhood_name == df_neighborhood_iptu.neighborhood),
)

In [183]:
# verifying the data integrity:
get_null_columns(df_neighborhood, normalize=True)  # looks good to go

-RECORD 0----------------------------------------------------------------------------------------------------------
 neighborhood_name                                                                          | 0.0                  
 neighborhood_alphabetized_population                                                       | 0.0                  
 neighborhood_area_in_hectares                                                              | 0.0                  
 neighborhood_average_age_household_leaders                                                 | 0.0                  
 neighborhood_average_age_women_household_leaders                                           | 0.0                  
 neighborhood_average_household_income                                                      | 0.0                  
 neighborhood_average_income_household_leaders                                              | 0.0                  
 neighborhood_average_income_women_household_leaders                    

# 2. Generating Layer-based Features

In [184]:
# registering dataframes to sql context:
df_neighborhood.createOrReplaceTempView("tb_neighborhood")
df_rail.createOrReplaceTempView("tb_stations")
df_parks.createOrReplaceTempView("tb_parks")
df_favelas.createOrReplaceTempView("tb_favelas")

## 2.1 Parks
Considering Parks as a feature takes into account two situations:
1. A region of any kind *containing* or *crossing* a park;
2. The distance of a region to the closest park; 

Since we are comparing polygons to polygons, it doesn't make much sense calculating the distance between them. Instead, we will summarize their units such that we can compare the distance between two points. In this case, we can use their centroids. 

> *Note: As of writing this notebook (July 4th, 2021), writing KNN queries (which is supported by Sedona's RDD api) can be made, but I will keep this for a later iteration of this project, as I am not that comfortable with the RDD api.*

In [185]:
# adding the centroid and area of the polygons:
Q_PARKS = """
SELECT 
    A.neighborhood_name,
    B.park_name,
    B.park_area
FROM tb_neighborhood as A, tb_parks as B
WHERE ST_Crosses(B.geometry, A.geometry)
"""

# matching the areas of ponderation to their sectors:
df_parks_match = spark.sql(Q_PARKS)

In [186]:
# grouping on district level to count the number of parks:
df_neighborhood_parks = df_parks_match.groupby("neighborhood_name").agg(
    F.countDistinct(F.col("park_name")).alias("n_parks"),
    F.sum(F.col("park_area")).alias("total_park_area"),
)

## 2.2 Favelas
To address Favelas as a feature, we have the same criteria as the case of Parks.

In [189]:
# adding the centroid and area of the polygons:
Q_FAVELAS = """
SELECT 
    A.neighborhood_name,
    B.favela_name,
    B.favela_area
FROM tb_neighborhood as A, tb_favelas as B
WHERE ST_Crosses(B.geometry, A.geometry)
"""

# matching the areas of ponderation to their sectors:
df_favelas_match = spark.sql(Q_FAVELAS)

In [190]:
# grouping on district level to count the number of parks:
df_neighborhood_favelas = df_favelas_match.groupby("neighborhood_name").agg(
    F.countDistinct(F.col("favela_name")).alias("n_favelas"),
    F.sum(F.col("favela_area")).alias("total_favela_area"),
)

## 2.3 Train Stations
Different from the previous cases, the train stations are represented as points. That makes it easier to match them to the level of interest at hand, since a point can be fully contained inside a polygon.

In [194]:
# adding the centroid and area of the polygons:
Q_STATIONS = """
SELECT 
    A.neighborhood_name,
    B.station_name
FROM tb_neighborhood as A, tb_stations as B
WHERE ST_Contains(A.geometry, B.geometry)
"""

# matching the areas of ponderation to their sectors:
df_stations_match = spark.sql(Q_STATIONS)

In [195]:
# grouping on district level to count the number of stations:
df_neighborhood_station = df_stations_match.groupby("neighborhood_name").agg(
    F.countDistinct(F.col("station_name")).alias("n_train_stations"),
)

# 3. Preparing the Layer-based features in the Neighborhood level

In [196]:
# joining layer-based features to the main dataset:
df_neighborhood_features = df_neighborhood.join(
    df_neighborhood_parks, on=["neighborhood_name"], how="left"
)

df_neighborhood_features = df_neighborhood_features.join(
    df_neighborhood_favelas, on=["neighborhood_name"], how="left"
)

df_neighborhood_features = df_neighborhood_features.join(
    df_neighborhood_station, on=["neighborhood_name"], how="left"
)

# dropping duplicates:
df_neighborhood_features = df_neighborhood_features.drop_duplicates(
    subset=["neighborhood_name"]
)

In [198]:
# fill null values in the new columns:
new_columns = [
    "n_parks",
    "total_park_area",
    "n_favelas",
    "total_favela_area",
    "n_train_stations",
]

df_neighborhood_features = df_neighborhood_features.fillna(0, subset=new_columns)

In [199]:
# adding a prefix for the level of interest to the new columns:
for col in new_columns:
    df_neighborhood_features = df_neighborhood_features.withColumnRenamed(
        col, f"neighborhood_{col}"
    )

# 4. Ranking-based columns
While it is valuable to have the aggregated features as raw values, it is also quite useful to have rank-based features, as they can be effectively a way to represent the categorical entities (the `district`, for example) directly as some sort of "embedding". I will generate some key features using this principle next using the `rank_feature` helper function, which, by default, transforms features into their `percent_rank` representation.

In [141]:
# adding rank on literacy rate -> the lowest literacy rate, the lower the rank
df_neighborhood_features = rank_feature(df_neighborhood_features, "neighborhood_literacy_rate")

In [142]:
# adding rank on monthly income
df_neighborhood_features = rank_feature(
    df_neighborhood_features, "neighborhood_average_monthly_income"
)

In [143]:
# adding rank on per capita income
df_neighborhood_features = rank_feature(
    df_neighborhood_features, "neighborhood_average_per_capita_income"
)

In [144]:
# adding rank on population (raw)
df_neighborhood_features = rank_feature(df_neighborhood_features, "neighborhood_population")

In [145]:
# adding rank on lot and construction square meter valuess
df_neighborhood_features = rank_feature(
    df_neighborhood_features, "neighborhood_average_lot_square_meter_value"
)

df_neighborhood_features = rank_feature(
    df_neighborhood_features, "neighborhood_average_construction_square_meter_value"
)

In [146]:
# adding rank on property_age -> the newer the properties, the higher the rank
df_neighborhood_features = rank_feature(
    df_neighborhood_features, "neighborhood_average_property_age", ascending=False
)

# 5. Exporting Neighborhood Features

In [205]:
# save the results to the specified directory:
NEIGHBORHOOD_OUTPUT = f"neighborhoods/tb_neighborhood"
FEATURE_UNITS_OF_INTEREST = "../data/feature/units_of_interest/"

df_neighborhood_features = df_neighborhood_features.drop("neighborhood_centroid")

save_geospatial_file(
    df_neighborhood_features,
    FEATURE_UNITS_OF_INTEREST + NEIGHBORHOOD_OUTPUT + ".parquet",
)

True

In [204]:
# saving the results without geometries:
NEIGHBORHOOD_NO_GEO = f"neighborhoods/tb_neighborhood_no_geo"

df_neighborhood_no_geo = df_neighborhood_features.drop("geometry")

save_to_filesystem(
    df_neighborhood_no_geo,
    FEATURE_UNITS_OF_INTEREST,
    NEIGHBORHOOD_NO_GEO,
    NEIGHBORHOOD_NO_GEO + ".parquet",
)

True